In [3]:
import numpy as np
from collections import defaultdict

import rospy
from std_msgs.msg import String
from geometry_msgs.msg import Point

In [4]:
class MonteCarloTreeSearchNode():
    def __init__(self, state, parent=None, parent_action=None):
        self.state = state
        self.parent = parent
        self.parent_action = parent_action
        self.children = []
        self._number_of_visits = 0
        self._results = defaultdict(int)
        self._results[1] = 0
        self._results[-1] = 0
        self._untried_actions = None
        self._untried_actions = self.untried_actions()
        return

    def untried_actions(self):

        self._untried_actions = self.state.get_legal_actions()
        return self._untried_actions

    def q(self):
        '''
        Update to reflect robot scenario rather than wins and losses
        '''
        wins = self._results[1]
        loses = self._results[-1]
        return wins - loses

    def n(self):
        return self._number_of_visits

    def expand(self):

        action = self._untried_actions.pop()
        next_state = self.state.move(action)
        child_node = MonteCarloTreeSearchNode(
            next_state, parent=self, parent_action=action)

        self.children.append(child_node)
        return child_node 

    def is_terminal_node(self):
        return self.state.is_game_over()

    def simulation(self):
        current_simulation_state = self.state

        while not current_simulation_state.is_game_over():

            possible_moves = current_simulation_state.get_legal_actions()

            action = self.simulation_policy(possible_moves)
            current_simulation_state = current_simulation_state.move(action)
        return current_simulation_state.game_result()

    def backpropagate(self, result):
        self._number_of_visits += 1.
        self._results[result] += 1.
        if self.parent:
            self.parent.backpropagate(result)

    def is_fully_expanded(self):
        return len(self._untried_actions) == 0

    def best_child(self, c_param=0.1):

        choices_weights = [(c.q() / c.n()) + c_param * np.sqrt((2 * np.log(self.n()) / c.n())) for c in self.children]
        return self.children[np.argmax(choices_weights)]

    def simulation_policy(self, possible_moves):

        return possible_moves[np.random.randint(len(possible_moves))]

    def _tree_policy(self):

        current_node = self
        while not current_node.is_terminal_node():

            if not current_node.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child()
        return current_node

    def best_action(self):
        simulation_no = 100


        for i in range(simulation_no):

            v = self._tree_policy()
            reward = v.simulation()
            v.backpropagate(reward)

        return self.best_child(c_param=0.)

    def get_legal_actions(self): 
        '''
        Modify according to your game or
        needs. Constructs a list of all
        possible actions from current state.
        Returns a list.
        '''

    def is_game_over(self):
        '''
        Modify according to your game or 
        needs. It is the game over condition
        and depends on your game. Returns
        true or false
        '''

    def game_result(self):
        '''
        Modify according to your game or 
        needs. Returns 1 or 0 or -1 depending
        on your state corresponding to win,
        tie or a loss.
        '''

    def move(self,action):
        '''
        Modify according to your game or 
        needs. Changes the state of your 
        board with a new value. For a normal
        Tic Tac Toe game, it can be a 3 by 3
        array with all the elements of array
        being 0 initially. 0 means the board 
        position is empty. If you place x in
        row 2 column 3, then it would be some 
        thing like board[2][3] = 1, where 1
        represents that x is placed. Returns 
        the new state after making a move.
        '''

def main():
    root = MonteCarloTreeSearchNode(state = initial_state)
    selected_node = root.best_action()
    return 

In [ ]:
class Robot():
    def __init__(self, robot_id, start_loc, goal_loc, time_interval):
        self.robot_id = robot_id
        self.start_loc = start_loc
        self.goal_loc = goal_loc
        self.loc_list = []
        self.observations_list = []
        self.time_interval = time_interval
        self.pub = rospy.Publisher('robot_loc_'+robot_id, Point, queue_size=10)
        rospy.init_node('robot_'+robot_id , anonymous=True)
        self.rate = rospy.Rate(1/time_interval)
        
    def get_observations(self):
        '''
        Return a dictionary of N,E,S,W and whether there
        is a wall in that direction from the robot's current 
        location
        '''
        
    def move(self, direction):
        '''
        Move one step in direction passed as argument
        if fail due to obstructing wall from get_observations()
        stay in same location
        '''
        
    def update(self):
        '''
        Move to next position, update observations, update locations, run MCTS (probably in new thread),
        publish to ROS topic
        '''      

In [ ]:
class Environment():
    def __init__(self, width, height, walls, start, goal, time_interval=0.5):
        self.width = width
        self.height = height
        self.walls = walls
        self.start = start
        self.goal = goal
        self.interval = time_interval
        self.timestep = 0
        self.complete = False
        self.robot_list = []
        
    def add_robot(self, robot_id, start_loc, goal_loc, time_interval):
        '''
        Add robot with start location start_loc and goal goal_loc and add to self.robot_list
        ''' 
        
    def render(self):
        '''
        render
        '''
        
    def listener(self):
        '''
        Implement listener code, call to renderer
        '''